In [1]:
import os
import sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


In [2]:
import json
import spacy
from pathlib import Path

from corpus_utils import *
from segmentation import *
from simple_structure import *
from dataset_utils import *


d:\Documents\nsu\ruArgMiner\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
RANDOM_STATE = 42
CORPORA_DIR = Path('../corpora')
OUTPUT_DIR = Path('../datasets')


In [4]:
with open(CORPORA_DIR / 'corpus_sci_comm.json') as corpus_file:
    corpus_sci_comm = Corpus.from_dict(json.loads(corpus_file.read()))

with open(CORPORA_DIR / 'corpus_sci_corpus.json') as corpus_file:
    corpus_sci_corpus = Corpus.from_dict(json.loads(corpus_file.read()))

flat_corpus_sci_comm = Corpus.flatten(corpus_sci_comm)
flat_corpus_sci_corpus = Corpus.flatten(corpus_sci_corpus)
full_corpus = Corpus.merge(flat_corpus_sci_comm, flat_corpus_sci_corpus)


'https://osf.io/qu4e2/download/ipl/#abc47ef1-8c9b-41ed-9c20-c13b124eb71a'
Excluding project "ISK" from text "Искусственный интеллект-3"
'https://osf.io/qu4e2/download/ipl/#c8099f8b-6c88-4717-9ebb-f0bde7938ddf'
Excluding project "ISK" from text "Искусственный интеллект-6"


In [5]:
spacy.prefer_gpu()
nlp = spacy.load('ru_core_news_lg')
nlp.enable_pipe('senter')

def project_with_most_comments(text):
    return max(text.projects, key=lambda project: len({stmt.record for stmt in project.statements}))

def segment_into_sentences(paragraphs):
    with nlp.select_pipes(enable=['tok2vec', 'parser']):
        return [[(sent.start_char, sent.end_char) for sent in doc.sents] for doc in nlp.pipe(paragraphs)]

def intersect_three_quarters_stmt(_, stmt_span, frag_span):
    return min(frag_span[1], stmt_span[1]) - max(frag_span[0], stmt_span[0]) >= 3/4 * (stmt_span[1] - stmt_span[0])

all_projects = [project_with_most_comments(text) for text in full_corpus.texts]
all_segmentations = segment_texts(full_corpus.texts, segment_into_sentences)
all_structures = [build_simple_structure(*z, intersect_three_quarters_stmt) for z in zip(all_projects, all_segmentations)]


In [6]:
dataset_kwargs = {
    'eval_size': 1/10,
    'test_size': 1/10,
    'random_state': RANDOM_STATE,
}

for ctx_len in [0, 1, 2]:
    build_dataset(all_structures, link_pairs_from_simple_structures, ctx_len=ctx_len, **dataset_kwargs).save_to_disk(OUTPUT_DIR / 'link_classification' / f'{ctx_len}ctx', max_shard_size='50MB')

link_classification_dataset_1ctx_label = build_dataset(all_structures, link_pairs_from_simple_structures, ctx_len=1, **dataset_kwargs).map(lambda row: {
    'fragment1_left':   f'Перед посылкой: {row["fragment1_left"]}',
    'fragment1':        f'Посылка: {row["fragment1"]}',
    'fragment1_right':  f'После посылки: {row["fragment1_right"]}',
    'fragment2_left':   f'Перед заключением: {row["fragment2_left"]}',
    'fragment2':        f'Заключение: {row["fragment2"]}',
    'fragment2_right':  f'После заключения: {row["fragment2_right"]}',
    'label': row['label'],
})
link_classification_dataset_1ctx_label.save_to_disk(OUTPUT_DIR / 'link_classification' / '1ctx_label', max_shard_size='50MB')


Saving the dataset (1/1 shards): 100%|██████████| 1428/1428 [00:00<?, ? examples/s]


In [7]:
for ctx_len in [0, 1]:
    triplet_dataset = build_dataset(all_structures, triplets_from_simple_structures, ctx_len=ctx_len, **dataset_kwargs)
    triplet_dataset_transitive = merge_datasets(triplet_dataset, build_dataset(all_structures, transitive_triplets_from_simple_structures, ctx_len=ctx_len, **dataset_kwargs))
    triplet_dataset.save_to_disk(OUTPUT_DIR / 'similarity_learning' / f'{ctx_len}ctx', max_shard_size='50MB')
    triplet_dataset_transitive.save_to_disk(OUTPUT_DIR / 'similarity_learning' / f'{ctx_len}ctx_transitive', max_shard_size='50MB')


Saving the dataset (1/1 shards): 100%|██████████| 3088/3088 [00:00<?, ? examples/s]


In [8]:
for ctx_len in [0, 1]:
    random_pairs_dataset = build_dataset(all_structures, random_pairs_from_simple_structures, ctx_len=ctx_len, **dataset_kwargs)
    random_pairs_dataset.save_to_disk(OUTPUT_DIR / 'random_pairs' / f'{ctx_len}ctx', max_shard_size='50MB')


Saving the dataset (1/1 shards): 100%|██████████| 14825/14825 [00:00<00:00, 29372015.49 examples/s]
